In [2]:
import pandas as pd
import csv
import re
from bs4 import BeautifulSoup

In [ ]:
df = pd.read_csv("src/archive/user_reviews.csv")

df = df.drop(["isVerified", "isSuperReviewer", "hasSpoilers", "hasProfanity", "userRealm"], axis=1)
df = df.groupby("userId").head(5)

df.to_csv("src/processed/user_reviews.csv", index=False)

In [ ]:
df = pd.read_csv("src/processed/user_reviews.csv")
df = df.drop(columns=['reviewId', 'score', 'userDisplayName'])

In [ ]:
# userId e movieId indicizzati con valori numerici
def create_id_mapping(df, column):
    unique_ids = df[column].unique()
    id_mapping = {id_: idx for idx, id_ in enumerate(unique_ids)}
    return id_mapping

# valori sostituiti nei campi originali
user_id_mapping = create_id_mapping(df, 'userId')
df['userId'] = df['userId'].map(user_id_mapping)

movie_id_mapping = create_id_mapping(df, 'movieId')
df['movieId'] = df['movieId'].map(movie_id_mapping)


# Salvataggio mapping dati originali-indicizzati
# # mapping per userId originale e indexed
# user_mapping_df = pd.DataFrame(list(user_id_mapping.items()), columns=['userId_original', 'userId_index'])
# user_mapping_df.to_csv('user_id_mapping.csv', index=False)

# # mapping per movieId originale e indexed
# movie_mapping_df = pd.DataFrame(list(movie_id_mapping.items()), columns=['movieId_original', 'movieId_index'])
# movie_mapping_df.to_csv('movie_id_mapping.csv', index=False)

In [ ]:
def remove_html_tags(text):
    """Rimuove i tag HTML da una stringa usando BeautifulSoup"""
    if text:
        return BeautifulSoup(text, "html.parser").get_text()
    return text

def clean_spaces_and_line_breaks(text):
    """Rimuove gli spazi bianchi e gli accapo inutili all'interno del testo"""
    if text:
        # Rimuoviamo gli spazi bianchi e gli accapo extra all'interno del testo
        # Lasciamo un solo spazio tra le parole e rimuoviamo quelli all'interno
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

def remove_quotes(text):
    """Rimuove tutte le virgolette (sia prima che dopo e all'interno del testo)"""
    if text:
        # Rimuoviamo le virgolette all'inizio e alla fine
        text = text.strip('"')
        # Rimuoviamo tutte le virgolette interne
        text = text.replace('"', '')
        return text
    return text

def clean_csv(input_file, output_file):
    """Pulisce il CSV rimuovendo i tag HTML, spazi bianchi, accapo in eccesso e virgolette"""
    
    with open(input_file, newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        rows = []
        current_row = None

        for row in reader:
            # Se la riga è vuota, la ignoriamo
            if not any(row):
                continue
            
            # Se la riga contiene solo una colonna, è una continuazione della precedente
            if len(row) == 1 and current_row is not None:
                # Aggiungiamo il testo della quote multilinea
                current_row[2] += " " + row[0]  # Supponiamo che la colonna quote sia la terza (indice 2)
            else:
                # Aggiungiamo la riga precedente completata
                if current_row:
                    # Rimuoviamo i tag HTML dalla colonna quote
                    current_row[2] = remove_html_tags(current_row[2])
                    current_row[2] = clean_spaces_and_line_breaks(current_row[2])  # Rimuoviamo spazi e accapo
                    current_row[2] = remove_quotes(current_row[2])  # Rimuoviamo le virgolette
                    rows.append(current_row)
                
                # Settiamo la nuova riga corrente
                current_row = row
        
        # Aggiungiamo l'ultima riga
        if current_row:
            current_row[2] = remove_html_tags(current_row[2])  # Rimuoviamo i tag HTML
            current_row[2] = clean_spaces_and_line_breaks(current_row[2])  # Rimuoviamo spazi e accapo
            current_row[2] = remove_quotes(current_row[2])  # Rimuoviamo le virgolette
            rows.append(current_row)
    
    # Scriviamo il nuovo file CSV pulito
    with open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(rows)

    print(f"File pulito salvato come '{output_file}'.")

# Esegui la pulizia del CSV
df.to_csv('user_reviews_input.csv', index=False)
input_file = 'user_reviews_input.csv'  # Sostituisci con il tuo file CSV di input
output_file = 'user_reviews_cleaned.csv'  # Sostituisci con il tuo file CSV di output
clean_csv(input_file, output_file)